# Student Details

**Name:** Shaunak A. Rai

**SRN:** PES2UG23CS546

**Date:** 26th February 2026

**Semester:** VI

**Branch:** CSE

**Section:** I

---

# Unit 2 Assignment: Building a Mixture of Experts (MoE) Router

## Objective
Build a **Smart Customer Support Router** using a Mixture of Experts (MoE) architecture with the Groq API.

The system routes user queries to specialized experts:
1. **Technical Expert** — for bug reports and code issues
2. **Billing Expert** — for refund/subscription/payment queries
3. **General Expert** — fallback for casual conversation

---

In [1]:
%pip install groq python-dotenv --quiet

from groq import Groq
from dotenv import load_dotenv
import os, time

load_dotenv()

client = Groq(api_key=os.environ.get("GROQ_API_KEY"))
MODEL = "llama-3.3-70b-versatile"

print(f"Groq client ready | Model: {MODEL}")


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Groq client ready | Model: llama-3.3-70b-versatile


## 1. Define the Experts

Each expert is defined by a specialized **System Prompt** that shapes the LLM's behavior. All experts use the same base model but behave differently based on their prompt.

In [2]:
MODEL_CONFIG = {
    "technical": {
        "system_prompt": (
            "You are a Senior Technical Support Engineer with deep expertise in software debugging. "
            "You are rigorous, code-focused, and precise. When a user reports a bug or technical issue, "
            "analyze the problem systematically, explain what went wrong, and provide concrete code fixes "
            "or step-by-step debugging instructions. Always include relevant code snippets."
        ),
        "temperature": 0.7
    },
    "billing": {
        "system_prompt": (
            "You are a Billing Support Specialist. You are empathetic, financially focused, and policy-driven. "
            "When a user has a billing concern, acknowledge their frustration first, clearly explain the relevant "
            "company policies, and outline the exact steps to resolve the issue such as refund process, "
            "escalation path, or account adjustments. Always be reassuring and professional."
        ),
        "temperature": 0.7
    },
    "general": {
        "system_prompt": (
            "You are a friendly and helpful General Support Agent. You handle casual questions, product inquiries, "
            "and anything that doesn't fall into technical or billing categories. Be conversational, helpful, "
            "and guide the user to the right resources if needed."
        ),
        "temperature": 0.7
    }
}

for name, cfg in MODEL_CONFIG.items():
    print(f"[{name.upper()}] temp={cfg['temperature']} | prompt_len={len(cfg['system_prompt'])} chars")

[TECHNICAL] temp=0.7 | prompt_len=349 chars
[BILLING] temp=0.7 | prompt_len=369 chars
[GENERAL] temp=0.7 | prompt_len=251 chars


## 2. The Router (`route_prompt`)

The router uses an LLM call with `temperature=0` to classify the user's intent into one of the expert categories. It returns **only** the category name.

In [3]:
def route_prompt(user_input):
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {
                "role": "system",
                "content": (
                    "You are a classification engine. Classify the user message into exactly one category: "
                    "technical, billing, or general. "
                    "Return ONLY the category name as a single lowercase word. No punctuation, no explanation."
                )
            },
            {"role": "user", "content": user_input}
        ],
        temperature=0,
        max_tokens=10
    )
    return response.choices[0].message.content.strip().lower()

test_routes = [
    "My python script is throwing an IndexError on line 5.",
    "I was charged twice for my subscription this month.",
    "Hey, what products do you offer?"
]

for query in test_routes:
    category = route_prompt(query)
    print(f"Query: {query!r}")
    print(f"  -> Routed to: [{category}]")
    print()
    time.sleep(1)

Query: 'My python script is throwing an IndexError on line 5.'
  -> Routed to: [technical]

Query: 'I was charged twice for my subscription this month.'
  -> Routed to: [billing]

Query: 'Hey, what products do you offer?'
  -> Routed to: [general]



## 3. The Orchestrator (`process_request`)

The orchestrator ties everything together:
1. Calls `route_prompt` to classify the query
2. Selects the matching expert configuration
3. Calls the LLM with the expert's system prompt
4. Returns the expert's response

In [4]:
def process_request(user_input):
    category = route_prompt(user_input)
    config = MODEL_CONFIG.get(category, MODEL_CONFIG["general"])

    print(f"[Router]  -> {category.upper()} Expert")
    print("-" * 50)

    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": config["system_prompt"]},
            {"role": "user", "content": user_input}
        ],
        temperature=config["temperature"],
        max_tokens=1024
    )

    answer = response.choices[0].message.content
    print(answer)
    return answer

## 4. Testing the MoE System

### Test 1: Technical Query

In [5]:
_ = process_request("My python script is throwing an IndexError on line 5. Here is the code: nums = [1,2,3]; print(nums[5])")

[Router]  -> TECHNICAL Expert
--------------------------------------------------
# Step-by-step analysis of the problem:
1. The code attempts to access the 6th element of the list `nums` (remember that indexing in Python starts at 0).
2. The list `nums` only contains 3 elements: `[1, 2, 3]`.
3. **The IndexError occurs because the index 5 is out of range** for the list `nums`, which only has indices 0, 1, and 2.

# Fixed solution:
```python
# Define the list of numbers
nums = [1, 2, 3]

# Check if the index is within the bounds of the list
index = 5
if index < len(nums):
    print(nums[index])
else:
    print(f"Index {index} is out of range for the list.")
```

# Explanation of changes:
* **Added a check to ensure the index is within the bounds of the list** before attempting to access the element at that index.
* Used the `len()` function to get the number of elements in the list.
* Provided a message to the user if the index is out of range.

# Tests and example uses:
* Try running th

### Test 2: Billing Query

In [6]:
time.sleep(2)
_ = process_request("I was charged twice for my subscription this month. I need a refund immediately.")

[Router]  -> BILLING Expert
--------------------------------------------------
I'm so sorry to hear that you were charged twice for your subscription this month. I can imagine how frustrating that must be for you, and I'm here to help resolve the issue as quickly as possible.

First, I want to assure you that we take situations like this very seriously and are committed to making it right. Our company policy is to provide a full refund for any duplicate charges that may have occurred due to an error on our part.

To initiate the refund process, I'll need to verify some information with you. Could you please confirm your subscription details, including the date of the duplicate charge and the amount that was charged? This will help me to locate the issue and process the refund promptly.

Once I have this information, I'll be able to provide a refund for the duplicate charge. Please note that refunds typically take 3-5 business days to process, but I'll do my best to expedite the process

### Test 3: General Query

In [7]:
time.sleep(2)
_ = process_request("Hey, what kind of products does your company offer?")

[Router]  -> GENERAL Expert
--------------------------------------------------
We have a wide range of products across various categories. We offer everything from home and kitchen essentials, to electronics, fashion, and outdoor gear. Whether you're looking for something for yourself or a gift for someone else, we've got you covered.

Some of our most popular products include smart home devices, gaming consoles, and high-quality cookware. We also have a great selection of clothing and accessories from top brands, as well as a variety of outdoor equipment for camping, hiking, and more.

If you're looking for something specific, I'd be happy to try and help you find it. Alternatively, you can also check out our website, which has a handy search function and categories to browse through. We're always adding new products to our lineup, so be sure to check back often to see what's new!

What kind of product are you in the market for? I'm here to help and can give you some recommendations i

---

## 5. Bonus: Tool Use Expert

Adding a **Tool Use** expert that intercepts queries asking for real-time data (like crypto prices) and routes them to a mock data-fetching function instead of the LLM.

In [8]:
def mock_fetch_price(asset):
    prices = {
        "bitcoin": 97432.15,
        "ethereum": 3521.80,
        "dogecoin": 0.1423,
        "solana": 189.67,
    }
    return prices.get(asset.lower())

MODEL_CONFIG["tool_use"] = {
    "system_prompt": "You are a Data Retrieval Agent. Extract real-time data using tools and present it clearly.",
    "temperature": 0.0
}

def route_prompt_v2(user_input):
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {
                "role": "system",
                "content": (
                    "You are a classification engine. Classify the user message into exactly one category: "
                    "technical, billing, tool_use, or general. "
                    "Use 'tool_use' if the user is asking for real-time data like prices, weather, or stocks. "
                    "Return ONLY the category name as a single lowercase word."
                )
            },
            {"role": "user", "content": user_input}
        ],
        temperature=0,
        max_tokens=10
    )
    return response.choices[0].message.content.strip().lower()

def process_request_v2(user_input):
    category = route_prompt_v2(user_input)
    print(f"[Router]  -> {category.upper()} Expert")
    print("-" * 50)

    if category == "tool_use":
        for asset in ["bitcoin", "ethereum", "dogecoin", "solana"]:
            if asset in user_input.lower():
                price = mock_fetch_price(asset)
                result = f"[Tool Fetch] The current price of {asset.title()} is ${price:,.2f} USD."
                print(result)
                return result
        print("[Tool Fetch] Asset not recognized. Falling back to general expert.")
        category = "general"

    config = MODEL_CONFIG.get(category, MODEL_CONFIG["general"])
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": config["system_prompt"]},
            {"role": "user", "content": user_input}
        ],
        temperature=config["temperature"],
        max_tokens=1024
    )
    answer = response.choices[0].message.content
    print(answer)
    return answer

print("MoE v2 with Tool Use ready.")

MoE v2 with Tool Use ready.


### Bonus Test: Tool Use

In [9]:
time.sleep(2)
_ = process_request_v2("What is the current price of Bitcoin?")
print()
time.sleep(1)
_ = process_request_v2("How much is Ethereum worth right now?")

[Router]  -> TOOL_USE Expert
--------------------------------------------------
[Tool Fetch] The current price of Bitcoin is $97,432.15 USD.

[Router]  -> TOOL_USE Expert
--------------------------------------------------
[Tool Fetch] The current price of Ethereum is $3,521.80 USD.
